In [4]:
from models.tda_models import TDA_PI34_Model
from models.raw_models import Raw_Model, Dummy_Model
from tda_pipelines import *


In [3]:
# fetch data, prepare for pipeline and test models

from sklearn.datasets import fetch_openml 
import numpy as np
from sklearn.model_selection import train_test_split

X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
print(f"X shape: {X.shape}, y shape: {y.shape}")
train_size, test_size = 1000, 300 # Reshape to (n_samples, n_pixels_x, n_pixels_y) 
X = X.reshape((-1, 28, 28)) 
X_train, X_test, y_train, y_test = train_test_split( X, y, train_size=train_size, test_size=test_size, stratify=y, random_state=666 ) 
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}") 
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


c:\Users\jawor\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


X shape: (70000, 784), y shape: (70000,)


In [6]:
tda_pipeline = TDA_PI34_Pipeline()

X_train_tda_clean = tda_pipeline.fit_transform(X_train)
print(X_train_tda_clean.shape)

X_test_tda_clean = tda_pipeline.transform(X_test)
print(X_test_tda_clean.shape)


In [8]:
# this needs to be integrated into pipeline

X_train_tda_clean_good = np.transpose(X_train_tda_clean, (0, 2, 3, 1))
X_test_tda_clean_good = np.transpose(X_test_tda_clean, (0, 2, 3, 1))


In [9]:
# mnist comes with string labels, we need to convert them to int

y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [11]:
# models

raw_model = Raw_Model()
dummy_model = Dummy_Model()
tda_model = TDA_PI34_Model()

# testing models below

In [13]:
tda_model.model.fit(X_train_tda_clean_good, y_train, epochs=20, batch_size=10, validation_data=(X_test_tda_clean_good, y_test))

Epoch 1/20


100/100 [==============================] - 7s 26ms/step - loss: 1.3545 - accuracy: 0.5660 - val_loss: 0.5917 - val_accuracy: 0.8467
Epoch 2/20
100/100 [==============================] - 2s 20ms/step - loss: 0.5853 - accuracy: 0.8460 - val_loss: 0.4673 - val_accuracy: 0.8700
Epoch 3/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3961 - accuracy: 0.8830 - val_loss: 0.4112 - val_accuracy: 0.8700
Epoch 4/20
100/100 [==============================] - 2s 16ms/step - loss: 0.3613 - accuracy: 0.8840 - val_loss: 0.3897 - val_accuracy: 0.8800
Epoch 5/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2291 - accuracy: 0.9350 - val_loss: 0.3495 - val_accuracy: 0.9100
Epoch 6/20
100/100 [==============================] - 2s 15ms/step - loss: 0.1830 - accuracy: 0.9360 - val_loss: 0.3669 - val_accuracy: 0.9167
Epoch 7/20
100/100 [==============================] - 2s 17ms/step - loss: 0.1327 - accuracy: 0.9620 - val_loss: 0.3410 - val_accuracy: 0.93

In [14]:
raw_model.model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/20
100/100 [==============================] - 6s 21ms/step - loss: 3.3624 - accuracy: 0.5210 - val_loss: 0.5701 - val_accuracy: 0.8067
Epoch 2/20
100/100 [==============================] - 1s 12ms/step - loss: 0.5751 - accuracy: 0.8100 - val_loss: 0.3828 - val_accuracy: 0.8600
Epoch 3/20
100/100 [==============================] - 2s 19ms/step - loss: 0.3057 - accuracy: 0.9010 - val_loss: 0.3385 - val_accuracy: 0.8833
Epoch 4/20
100/100 [==============================] - 1s 14ms/step - loss: 0.2451 - accuracy: 0.9200 - val_loss: 0.4477 - val_accuracy: 0.8767
Epoch 5/20
100/100 [==============================] - 2s 23ms/step - loss: 0.2095 - accuracy: 0.9300 - val_loss: 0.2816 - val_accuracy: 0.9100
Epoch 6/20
100/100 [==============================] - 1s 14ms/step - loss: 0.0938 - accuracy: 0.9710 - val_loss: 0.4197 - val_accuracy: 0.9067
Epoch 7/20
100/100 [==============================] - 2s 18ms/step - loss: 0.0342 - accuracy: 0.9910 - val_loss: 0.5050 - val_accuracy: 0.8833

In [15]:
dummy_model.model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/20
100/100 [==============================] - 4s 13ms/step - loss: 19.7019 - accuracy: 0.5820 - val_loss: 7.6441 - val_accuracy: 0.7667
Epoch 2/20
100/100 [==============================] - 1s 8ms/step - loss: 3.1578 - accuracy: 0.8470 - val_loss: 6.3984 - val_accuracy: 0.7933
Epoch 3/20
100/100 [==============================] - 1s 7ms/step - loss: 1.8467 - accuracy: 0.8910 - val_loss: 6.4596 - val_accuracy: 0.7800
Epoch 4/20
100/100 [==============================] - 1s 8ms/step - loss: 0.8523 - accuracy: 0.9270 - val_loss: 5.6980 - val_accuracy: 0.8100
Epoch 5/20
100/100 [==============================] - 1s 10ms/step - loss: 1.2152 - accuracy: 0.9300 - val_loss: 6.0334 - val_accuracy: 0.8133
Epoch 6/20
100/100 [==============================] - 1s 12ms/step - loss: 0.6156 - accuracy: 0.9550 - val_loss: 5.7768 - val_accuracy: 0.7867
Epoch 7/20
100/100 [==============================] - 1s 10ms/step - loss: 1.0526 - accuracy: 0.9310 - val_loss: 4.9284 - val_accuracy: 0.7967
E